# Funciones Acumulativas

Las funciones acumulativas nos permiten trabajar en la fila "n" de una tabla con todos los datos de las filas "0 a n"
<br>Dentro de este tipo de funciones encontramos:

* cummax()  El máximo acumulado (ideal para máximo histórico por fecha)
* cummin()  El mínimo acumulado (ideal para mínimo histórico por fecha)
* cumsum() La suma acumulada (ideal para armado se subtotales por fecha)
* cumprod() El producto acumulado (ideal para rendimiento compuesto)

## Cummax()

El uso típico que le vamos a dar a esta función es para saber el máximo histórico de una serie en cada punto de la misma

In [ ]:
import pandas as pd
data = pd.read_excel('excels_csvs/AAPL.xlsx')
data = data.sort_values(by='timestamp',ascending=True)
data = data.drop(["high","low","volume"], axis=1).set_index("timestamp")

data['maxHist'] = data.adjusted_close.cummax()
data.head(6)

## Cummin()

Obviamente es lo mismo que el cummax pero para mínimos, la combinación de cummax() y cummin() va a ser muy util para backtestings de drawdowns y recuperaciones posteriores

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx')
data = data.sort_values(by='timestamp',ascending=True)
data = data.drop(["high","low","volume"], axis=1).set_index("timestamp")

data['minHist'] = data.adjusted_close.cummin()
data.head(6)

## Cumsum()

Cumsum() es obviamente una funcion de sumas acumuladas, que en la posicion de la fila "n" nos devuelve la suma de "0 a n" (inclusive)

$$ \large cumsum\hspace{3mm}(X_{n})\hspace{3mm}  =  \hspace{3mm} \sum_{i=0}^{n}x_{i} $$

En el ejemplo aprovechamos para borrar las columnas de OHLC par limpiar un poco la salida con la función drop()

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx')
data = data.sort_values(by='timestamp',ascending=True)

data['volumenAcum'] = data.volume.cumsum()/1000000
data = data.drop(['open','high','low','close'],1).dropna().round(2)
data.set_index('timestamp',inplace=True)
data.head(6)

In [ ]:
data.volumenAcum.plot()

## Cumprod()

Cumprod() es una función de productorio, es decir el producto acumulado de 0 a n, para la fila n 

$$ \large cumprod\hspace{3mm}(X_{n})\hspace{3mm}  =  \hspace{3mm} \prod_{i=0}^{n}x_{i} $$

Vamos a usar esta fórmula para calcular rendimiento compuesto, es muy sencillo:
<br>  1- Creamos una columna "variacion" con el valor "r", rendimiento porcentual diario.
<br>  2- Creamos una columna "factor" con el valor (1+r)
<br>  2- Luego vamos a aplicar el productorio para cada fila de esa columna "factor" y le restamos 1 al resultado.

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx')
data = data.sort_values(by='timestamp',ascending=True)

data['variacion'] = data['adjusted_close'].pct_change()
data['factor'] = 1 + data['variacion'] 
data['rendimientoAcum'] = (data.factor.cumprod()-1)*100

data = data.drop(['open','high','low','close','volume'],1).dropna().round(4).set_index("timestamp")
data

In [ ]:
dataf = data.iloc[:250]
dataf.rendimientoAcum.plot()

## Rolling vs Acumulativas (diferencias)
* cummax vs rolling(n).max()
* cummin vs rolling(n).min()
* cumsum vs rolling(n).sum()



# Agrupamiento

In [ ]:
import pandas as pd

data = pd.read_excel('excels_csvs/SPY.xlsx')
data = data.sort_values('timestamp',ascending=True).set_index("timestamp")
data

## Agrupamiento por columnas o toda la tabla

In [ ]:
data = pd.read_excel('excels_csvs/SPY.xlsx')
data = data.sort_values('timestamp',ascending=True).set_index("timestamp")
data['variacion'] = data.adjusted_close.pct_change()

pd.options.display.max_rows = 10

agrupados = data.volume.groupby(data.index.dayofweek).sum()


agrupados

In [ ]:
data = pd.read_excel('excels_csvs/SPY.xlsx')
data = data.sort_values('timestamp',ascending=True).set_index("timestamp", drop=False)
data['variacion'] = data.adjusted_close.pct_change()
agrupados = data.adjusted_close.groupby(data.index.to_period('Q')).last()

agrupados

## Métodos para concatenar a los agrupamientos

Así como usamos la funcón count() podemos usar otro tipo de funciones, por ejemplo:
* first()
* last()
* min()
* max()
* sum()
* prod()
* mean()
* median()
* std() desvio estandar (sigma)
* var() varianza (sigma^2)
* skew()  (Coeficiente de asimetria)
* kurtosis() 
* quantile()  

## Criterios de agrupamiento del tipo fechas

# Mas funciones

## Clip (Acotado)
Esta función "acota" los valores de una columna entre un mínimo y un máximo, es muy útil para descartar la incidencia de "outliers" o "datos aberrantes" o "valores de colas" o simplemente "errores" del data feed

In [ ]:
tabla = pd.read_excel('excels_csvs/AAPL_SPY_QQQ.xlsx')
tabla.set_index("timestamp",inplace=True)
tabla.tail(8)

Supongamos que no queremos considerar los valores superiores a +8% o los inferiores a -8% y queremos "topearlos" en esos límites

In [ ]:
tabla = pd.read_excel('excels_csvs/AAPL_SPY_QQQ.xlsx')    
tabla.set_index("timestamp",inplace=True)
tabla_acotada = tabla.clip(-6,8)
tabla_acotada.tail(8)

como vemos en el dato del 2 de marzo de 2020, que teníamos un 9.31, lo cambió por 8 que era nuestro "tope"

## Funciones Estadísticas Báscias
Gracias a pandas tenemos a disposición de un clic las siguientes funcones estadísticas básicas:
* Valor máximo: max()
* Valor mínimo: min()
* Indice de valores mínimos y máximos: idxmin() e idxmax()
* Media: mean()
* Mediana: madian()
* Producto: prod()  o product()
* Suma: sum()
* Ranking: rank()
* Quantiles: quantile()
* Cantidad de valores únicos: nunique()

Tambien podemos agrupar por otro tipo de datos:
* Cualquier columna discreta (supongamos que tengamos una con True/False)
* Tambíen se puede discretizar un rango continuo pero ya veremos ottras herramientas mas interesantes para eso
* Cualquier otro agrupamiento de fechas:
    * year
    * month
    * week
    * dayofweek
    * mas de un criterio
    * Trimestral: usando to_period('Q') 

# Repaso de Estadística

## Desvío Estandrar - Fórmulas

$$ \large \sigma^2 = \sum_{x=i}^{n} \frac{1}{n} . (X_i - \bar{X} )^2  $$

In [ ]:
data.variacion.std()

## Error estandar - Fórmulas

$$ \Large \frac{\sigma}{\sqrt{n}}$$ 

In [ ]:
data.variacion.sem()

## Varianza

In [ ]:
data.variacion.var()

## Skew - Fórmulas

$$ \large  skew =  \frac{\mu_3}{\sigma^3}  =  \frac{\sum_{x=i}^{n} \frac{1}{n} . (X_i - \bar{X} )^3}{\sigma^3} $$

## Repaso de Coeficiente de Asimetría

<img src='imagenes/skew.png' style='width:600px;float:left;'>

## Repaso de quantiles, cuartiles, quintiles, deciles, percentiles y blabla_iles

<img src='imagenes/quantiles.png' style='width:600px;float:left;'>

In [ ]:
data.variacion.quantile(0.5)

In [ ]:
data.variacion.rank(pct=True)

In [ ]:
data.variacion.quantile(0.048853)

In [ ]:
data.variacion['2000-03-07']

## Kurtosis

$$ \Large kurtosis = \frac{\sum_{i=1} (X_i-\bar{X})^4}{n . \sigma^4}$$


Coeficiente de apuntamiento

<div style="width:600px;float:left;">
    <br>La Curtosis nos da una idea de la forma, mientras mas alta, mas valores cerca de la media y mas gruesas las colas
    <ul>
        <li> Leptocúrtica,  Curtosis > 3 (más apuntada y con colas más gruesas que la normal) </li>
        <li> Platicúrtica,  Curtosis < 3 (menos apuntada y con colas menos gruesas que la norma) </li>
        <li> Mesocúrtica,   Curtosis = 3 (tiene una distribución normal) </li>
    </ul>
</div>
<div style="width:350px;float:right;margin-top:-25px;">
    <img src="imagenes/kurtosis.png" style="width:300px;height:230px;">
</div>


In [ ]:
data.variacion.kurtosis()

## Covarianza

$$ \large s_{xy}={1 \over n}\sum _{i=1}^{n}{(x_{i}-{\overline {x}})(y_{i}-{\overline {y}})} $$

In [ ]:
import yfinance as yf

activos = ['GGAL','YPF','PAM','EDN','BBAR']
data = yf.download(activos, start='2001-01-01', end='2020-08-30')['Adj Close']
tabla = data.pct_change().dropna()*100
tabla

### Covarianza contra si mismo

In [ ]:
tabla.var()

In [ ]:
tabla.GGAL.cov(tabla.GGAL)

### Matriz de covarianzas

In [ ]:
tabla.cov()

# Método apply en lugar de concatenar métodos

In [ ]:
data = pd.read_excel('excels_csvs/SPY.xlsx')
data = data.sort_values('timestamp',ascending=True).set_index("timestamp", drop=False)
data['variacion'] = data.adjusted_close.pct_change()

data.variacion.groupby(data.index.year).apply(pd.DataFrame.std)

In [ ]:
data = pd.read_excel('excels_csvs/SPY.xlsx')
data = data.sort_values('timestamp',ascending=True).set_index("timestamp", drop=False)
data['variacion'] = data.adjusted_close.pct_change()

data.variacion.groupby(data.index.year).std()

# Ejemplos de concatenado con groupby

In [ ]:
data.variacion.groupby(data.index.year).quantile(0.75)

In [ ]:
data.variacion.groupby(data.index.year).describe()

In [ ]:
data = yf.download('SPY')
data['variacion'] = data['Adj Close'].pct_change()
data.variacion.groupby(data.index.year).apply(pd.DataFrame.kurtosis).plot()

In [ ]:
data = yf.download('KO', start='1990-01-01')
data['variacion'] = data['Adj Close'].pct_change()
data.variacion.groupby(data.index.year).apply(pd.DataFrame.kurtosis).plot()

### Agrupamiento de booleanos y discretos (Atencion a esto)

De paso vemos el WHERE() en un dataFrame

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index("timestamp").sort_index()

data['intradiario']  = data.close - data.open
data['cierre_previo']=data['close'].shift()
data['gap_positivo']=data.eval('open>cierre_previo')

# Lo defino por default arbitrariamente
data['vela_color'] = 'Verde'  

# Ahora si pregunto, si se da la condicion dejo el default si no la cambio:
data['vela_color']= data.vela_color.where(data.close > data.open,"Roja")

data

#### Agrupando por SIZE o COUNT: Diferencia

In [ ]:
data.groupby('gap_positivo').mean()

In [ ]:
data.groupby('gap_positivo').intradiario.count()

In [ ]:
data.groupby('gap_positivo').size()

In [ ]:
data.groupby('gap_positivo').size().reset_index(name='total')

In [ ]:
data.groupby(['gap_positivo','vela_color']).size()

# Combinacion de Agrupamiento + Filtros + Ordenamientos

## Agrupamiento con filtros

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp').sort_index()

# Calculo las variaciones
data['variaciones'] = data['adjusted_close'].pct_change()*100

# Filtramos las variaciones > 10%
filtro = data.loc[ data['variaciones'] > 10 ]  

# Contabilizamos por año ese filtro
agrupados = filtro.variaciones.groupby(filtro.index.year).count().to_frame()
agrupados.columns = ['Subas +10%']
agrupados

## ¿Y al reves? primero agrupamiento y luego filtro? ¿cuando usar cada variante?

Ejemplo: Ver semanas con rendimiento mayor al 13% (ponele)

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp').sort_index()
data['factor']=data.adjusted_close.pct_change()+1

yields = pd.DataFrame()
yields['Yield'] = (data.factor.groupby([data.index.year, data.index.week]).prod() -1)*100
filtro = yields.loc[yields.Yield >13]
filtro

## ¿Y Combinar agrupamiento y ordenamiento? ¿para que sirve esta combinación?

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp').sort_index()
data['factor']=data.adjusted_close.pct_change()+1

yields = pd.DataFrame()
yields['Yield'] = (data.factor.groupby([data.index.year, data.index.month]).prod() -1)*100
ordenado = yields.sort_values("Yield", ascending=False)
ordenado.index.names = ['Año','Semana']
ordenado.head(10)

# Resampleo

Mediante el metodo resample podemos reagrupar rapidamente en funcion de diferentes timeframes una serie dada
<br><b>Es importante aclarar que para que funcione el resampleo el indice de la tabla debe ser el timestamp</b>

Las Frecuencias posibles son
* B = business day frequency
* D = calendar day frequency
* W = weekly frequency 
* M = month end frequency 
* BM = business month end frequency 
* MS = month start frequency 
* BMS = business month start frequency 
* Q = quarter end frequency 
* BQ = business quarter endfrequency 
* QS = quarter start frequency 
* BQS = business quarter start frequency 
* A = year end frequency 
* BA = business year end frequency 
* AS = year start frequency 
* BAS = business year start frequency 
* BH = business hour frequency 
* H = hourly frequency 
* T = minutely frequency 
* S = secondly frequency 
* L = milliseonds

## Jugando con la fecha de resampleo y la de la muestra.. ojo ahi

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp', drop=False).sort_index()

resampleada = data.resample('10T').last()
resampleada.dropna()

## Cierres al ultimo dia HABIL del mes de cada año

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp', drop=False).sort_index()

resampleada = data.resample('BM').last()
resampleada.head(12)

## Cierres cada bimestre

Ojo acá, si ponemos una cantidad "X" antes del tipo de resampleo, me arma la primera muestra y a partir de ella usa esa cantidad

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel('excels_csvs/AAPL.xlsx').set_index('timestamp', drop=False).sort_index()

cierresBimestrales = data['adjusted_close'].resample('2M').last().to_frame()
cierresBimestrales

## Trabajando con intervalos de minutos

Veamos promero que me levanta del archivo:

In [ ]:
data = pd.read_excel('excels_csvs/AAPL_INTRA.xlsx').set_index('datetime', drop=False).sort_index()
data.head(6)

Como vemos partimos de una serie cada 2 minutos, vamos a resamplearla cada 10 minutos:

In [ ]:
res1 = data['adjusted_close'].resample('10T').last().to_frame()
res1.head()

Acá la resampleamos cada 1 día
<br>Ojo al poner first() tomamos el primer valor de dia, obviamente si ponemos last() tomariamos el ultimo
<br>Del mismo modo si ponemos mean() tomariamos el valor medio

In [ ]:
res2 = data['adjusted_close'].resample('1D').first().to_frame()
res2.columns = ["Apertura"]
res2.head()

Como ven aparecen días sin datos, claramente son feriados y/o sábados y domingos, para sacar esos datos, como ya vimos podemos usar la funcion dropna()

In [ ]:
res2 = data['adjusted_close'].resample('1D').last().to_frame().dropna()
res2.columns = ["Cierre"]
res2.head()

## Armando una tabla de varias columnas con un resampleo

In [ ]:
tabla = data['adjusted_close'].resample('D').mean().to_frame().dropna()
tabla['last'] = data['adjusted_close'].resample('D').last().to_frame()
tabla['first'] = data['adjusted_close'].resample('D').first().to_frame()

tabla.columns = ["Precio Medio","Precio Cierre",'Precio Apertura']

tabla.round(2).head()

# Ajuste de series (ej x gaps)

In [ ]:
import pandas as pd
import analisis_tecnico as at

In [ ]:
data = at.getDataExcel('AAPL')

# Funcion de ajuste de datos
data = at.ajustarOHLC(data)


data['pctChange'] = data.AdjClose.pct_change()
data['Price'] = data.AdjClose
data['Mov']=data.AdjClose.pct_change()*100
data['OpenGap']=(data.Open/data.Close.shift(1)-1)*100
data['IntraMov']=(data.Close/data.Open-1)*100
data.index.name = "Date"

data.dropna(inplace=True)
data

In [ ]:
data.OpenGap.plot()

In [ ]:
data.sort_values('OpenGap').head()

In [ ]:
import matplotlib.pyplot as plt

series = [data.OpenGap,data.IntraMov]
fig, ax = plt.subplots(figsize=(4,6))

ax.boxplot(series, showmeans=True)
ax.set_ylim([-15,15])

plt.xticks([1,2],["GAPS","Intra"])

plt.show()

# Graficos superpuestos

## Ejemplo con pandas directo

In [ ]:
data = at.getDataExcel('AAPL')

# Funcion de ajuste de datos
data = at.ajustarOHLC(data)

data['pctChange'] = data.AdjClose.pct_change()
data['Price'] = data.AdjClose
data['Mov']=data.AdjClose.pct_change()*100
data['OpenGap']=(data.Open/data.Close.shift(1)-1)*100
data['IntraMov']=(data.Close/data.Open-1)*100
data.index.name = "Date"
data.dropna(inplace=True)

In [ ]:
test = data.loc[:,['OpenGap','IntraMov']]
test = (test/100 +1).dropna()


tabla = test.groupby([test.index.year,test.index.month]).prod()-1
tabla.rolling(12).mean().plot()

plt.plot([0]*len(tabla), 'k--')

In [ ]:
tabla

In [ ]:
tabla.abs().sum()

## Ejemplo con matplotlib

In [ ]:
import yfinance as yf
df = yf.download('GGAL')
df['variacion'] = df['Adj Close'].pct_change()
df['volatilidad'] = df['variacion'].rolling(250).std() * 250**0.5
df.dropna(inplace=True)
df

## Escalas diferentes?

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,6))

ax.plot(df['Adj Close'], 'r-', label='Precio')
ax.plot(df['volatilidad'], 'b--', label='volatilidad')

plt.legend()
plt.show()

## Escalas diferentes

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(df['Adj Close'], 'r-',  label='Precio')
ax.legend(loc='upper left')

ax2 = ax.twinx()
ax2.plot(df['volatilidad'], 'k--',  label='volatilidad')
ax2.legend(loc='upper right')

ax2.legend()

plt.show()


## Escalas lineales y logaritmicas juntas

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(df['Adj Close'], 'g:',label='Precio Log')
ax.set_yscale('log')
ax.legend(loc='upper left', fontsize=14)

ax2 = ax.twinx()
ax2.plot(df['Adj Close'], 'r-',label='Precio Lineal')
ax2.legend(loc='upper right', fontsize=14)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(df['Adj Close'], 'g:',label='Precio Log')
ax.set_yscale('log')
ax.legend(loc='upper left', fontsize=14)

ax2 = ax.twinx()
ax2.plot(df['Adj Close'], 'r-',label='Precio Lineal')
ax2.set_ylim(0,100)
ax2.legend(loc='upper right', fontsize=14)

plt.show()

# Subplots

## Forma antigua

In [ ]:
import analisis_tecnico as at

data = at.getDataExcel('AAPL')

# Funcion de ajuste de datos
data = at.ajustarOHLC(data)

data['pctChange'] = data.AdjClose.pct_change()
data['Price'] = data.AdjClose
data['Mov']=data.AdjClose.pct_change()*100
data['OpenGap']=(data.Open/data.Close.shift(1)-1)*100
data['IntraMov']=(data.Close/data.Open-1)*100
data.index.name = "Date"
data.dropna(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))

# asigno a la variable ax1 un subplot
# el 211 significa 2 filas, 1 columa, ocupar el 1° lugar
ax1 = plt.subplot(221)
ax1.plot(data.Price)
ax1.set_yscale('log')

# asigno a la variable ax2 otro subplot
# el 212 significa 2 filas, 1 columa, ocupar el 2° lugar
ax2 = plt.subplot(222)
ax2.plot(data.Price)
ax2.set_yscale('linear')

## Forma nueva

In [ ]:
fig, axs = plt.subplots(2, figsize=(10,7))
axs[0].plot(data.Price)
axs[0].set_yscale('log')
axs[0].title.set_text('Escala Logaritmica')
axs[1].plot(data.Price)
axs[1].set_yscale('linear')
axs[1].title.set_text('Escala Lineal')
fig.subplots_adjust(hspace=0.3)

Les dejo exactamente lo mismo pero apilados horizontalmente, solo cambia la línea que define los subplots(filas, columnas) que en lugar de poner subplots(2) (que indica 2 filas) ponemos subplot(1,2) que indica una fila 2 columnas

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))
axs[0].plot(data.Price)
axs[0].set_yscale('log')
axs[0].title.set_text('Escala Logaritmica')
axs[1].plot(data.Price)
axs[1].set_yscale('linear')
axs[1].title.set_text('Escala Lineal')
fig.subplots_adjust(hspace=0.4)

Les dejo como ejemplo un básico de precio y volumen

In [ ]:
data = at.getDataExcel('AAPL')
precios2020 = data.loc[(data.index>='2019')]

fig, axs = plt.subplots(2, figsize=(10,5), gridspec_kw={'height_ratios':[2, 1]})
fig.suptitle('Serie AAPL con Volumen', y= 0.94, color='black')

axs[0].plot(precios2020.index, precios2020.AdjClose)
axs[1].bar(precios2020.index, precios2020.Volume)
fig.subplots_adjust(hspace=0)

# Estilos predeterminados

Hay varios estilos predeterminados de colores y tamaños, les dejo el comando para listarlos a todos, después vayan ustedes probando cual les gusta mas, el paquete de estilos se llama con es plt.style y pueden listarlos con plt.style.available o bien pueden seleccionar uno con plt.style.use("nombre_del_estilo_elegido")

In [ ]:
print(plt.style.available)

A continuación dejo algunos ejemplos solo para mostrar que solo cambiando esa línea ya se puede elegir una variedad de estilos bastante diferenciados

In [ ]:
import pandas as pd
import analisis_tecnico as at
import matplotlib.pyplot as plt


data = at.getDataExcel('AAPL')
data = at.ajustarOHLC(data)
data = at.addGap(data)


plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(7,3))
plt.gca().set_yscale('log')
plt.plot(data.Price)

In [ ]:
plt.style.use('fivethirtyeight')

plt.figure(figsize=(7,3)).suptitle('fivethirtyeight')
plt.gca().set_yscale('log')
plt.plot(data.Price)

Supongamos que me encantó ese diseño pero no quiero que dibuje las grillas
<br>En ese caso hago la captura del objeto ejes y le aplico el método grid() y lo configuro como False

In [ ]:

plt.style.use('fivethirtyeight')
plt.figure(figsize=(7,3)).suptitle('fivethirtyeight')

plt.gca().set_yscale('log')
plt.gca().grid(False)

plt.plot(data.Price)

Aplicamoos otro estilo, usemos el seaborn

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize=(7,3)).suptitle('seaborn')
plt.gca().set_yscale('log')
plt.plot(data.Price)

Pero supongamos que me encantó ese estilo y no me gusto ese color de fondo de los ejes
<br>Entonces podemos aplicar el método set_facecolor("color") a los ejes que debemos capturar

In [ ]:
plt.style.use('seaborn')

plt.figure(figsize=(7,3)).suptitle('seaborn')
plt.gca().set_yscale('log')
plt.gca().set_facecolor("white")
plt.plot(data.Price)

### Gráficos con dos Ejes Y diferentes y el mismo eje X

In [ ]:
import pandas as pd
import analisis_tecnico as at
import matplotlib.pyplot as plt


data = at.getDataExcel('AAPL')
data = at.ajustarOHLC(data)
data = at.addGap(data)
data['STD'] = data.pctChange.rolling(50).std()

fig, ax1 = plt.subplots(figsize=(10,4))
plt.style.use('fivethirtyeight')

fig.suptitle('Precios y Volatilidad SPY')

ax1.set_ylabel('Precios', color="k")
ax1.set_yscale('log')
ax1.plot(data.Price, lw=1, color="k")

ax2 = ax1.twinx()  
ax2.set_ylabel('Volatilidad', color="gray")  
ax2.plot(data.STD, lw=1.5, linestyle="--", color="gray")